# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [3]:
# Import initial libraries
import numpy as np
import pandas as pd
from citipy import citipy

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [17]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90,90,2000)
lngs = np.random.uniform(-180,180,2000)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats,lngs)

In [18]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)
print(coordinates[0:3])

[(3.696697591910251, 83.87307714086302), (-14.999973774640978, 104.20251946200739), (-89.78051086861137, -131.76520257053218)]


## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [6]:
# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

768

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [7]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [12]:
# Assemble the OpenWeatherMap's API call

url = f"https://api.openweathermap.org/data/2.5/weather?lat={lats}&lon={lngs}&appid=" + weather_api_key
print(url)

https://api.openweathermap.org/data/2.5/weather?lat=[-53.12740305 -37.69639827  80.7712465  ... -63.02038929  55.24247772
 -48.15873876]&lon=[-100.66830759  117.92372394  -57.22629156 ...  -84.0191598  -173.17899111
  -15.43952891]&appid=90d5d0f1ccb6f9a40fcbcf02a8a80631


In [19]:
# Preview city data
cities[0:5]

['castro', 'albany', 'upernavik', 'ushuaia', 'barrow']

In [20]:
r = requests.get(url + "&q=" + cities[13].replace(" ", "+")).json()

In [21]:
r

{'coord': {'lon': -113.5841, 'lat': 37.1041},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 297,
  'feels_like': 296.54,
  'temp_min': 296.92,
  'temp_max': 297.56,
  'pressure': 1019,
  'humidity': 42},
 'visibility': 10000,
 'wind': {'speed': 6.69, 'deg': 120},
 'clouds': {'all': 0},
 'dt': 1664849345,
 'sys': {'type': 2,
  'id': 2008042,
  'country': 'US',
  'sunrise': 1664803881,
  'sunset': 1664846105},
 'timezone': -21600,
 'id': 5546220,
 'name': 'Saint George',
 'cod': 200}

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [22]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']      
            
        # Append the city information into the city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_weather_description})
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | castro
Processing Record 2 of Set 1 | albany
Processing Record 3 of Set 1 | upernavik
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | barrow
Processing Record 6 of Set 1 | merauke
Processing Record 7 of Set 1 | bluff
Processing Record 8 of Set 1 | rikitea
Processing Record 9 of Set 1 | chokurdakh
Processing Record 10 of Set 1 | vaini
Processing Record 11 of Set 1 | mataura
Processing Record 12 of Set 1 | cidreira
Processing Record 13 of Set 1 | whitehorse
Processing Record 14 of Set 1 | saint george
Processing Record 15 of Set 1 | toliary
City not found. Skipping...
Processing Record 16 of Set 1 | torbay
Processing Record 17 of Set 1 | watsa
Processing Record 18 of Set 1 | avera
Processing Record 19 of Set 1 | saint-philippe
Processing Record 20 of Set 1 | hanzhong
Processing Record 21 of Set 1 | souillac
Processing Record 22 of Set 1 | butaritari
Processing Record 23 of Set

Processing Record 41 of Set 4 | belmonte
Processing Record 42 of Set 4 | mamallapuram
Processing Record 43 of Set 4 | husavik
Processing Record 44 of Set 4 | luanda
Processing Record 45 of Set 4 | nizhneyansk
City not found. Skipping...
Processing Record 46 of Set 4 | hithadhoo
Processing Record 47 of Set 4 | saint-augustin
Processing Record 48 of Set 4 | mildura
Processing Record 49 of Set 4 | senmonorom
City not found. Skipping...
Processing Record 50 of Set 4 | axim
Processing Record 1 of Set 5 | luderitz
Processing Record 2 of Set 5 | victoria point
Processing Record 3 of Set 5 | yulara
Processing Record 4 of Set 5 | kavieng
Processing Record 5 of Set 5 | dzhusaly
City not found. Skipping...
Processing Record 6 of Set 5 | zenzeli
Processing Record 7 of Set 5 | onda
Processing Record 8 of Set 5 | lubango
Processing Record 9 of Set 5 | labuhan
Processing Record 10 of Set 5 | campoverde
Processing Record 11 of Set 5 | povenets
Processing Record 12 of Set 5 | yellowknife
Processing Rec

Processing Record 28 of Set 8 | galle
Processing Record 29 of Set 8 | tokmak
Processing Record 30 of Set 8 | sitka
Processing Record 31 of Set 8 | san cristobal
Processing Record 32 of Set 8 | khvoynaya
Processing Record 33 of Set 8 | herat
Processing Record 34 of Set 8 | nioro
Processing Record 35 of Set 8 | fallon
Processing Record 36 of Set 8 | ballina
Processing Record 37 of Set 8 | port hardy
Processing Record 38 of Set 8 | pokhara
Processing Record 39 of Set 8 | kasama
Processing Record 40 of Set 8 | northam
Processing Record 41 of Set 8 | christchurch
Processing Record 42 of Set 8 | alpignano
Processing Record 43 of Set 8 | san luis
Processing Record 44 of Set 8 | severnoye
Processing Record 45 of Set 8 | flin flon
Processing Record 46 of Set 8 | sistranda
Processing Record 47 of Set 8 | riviere-au-renard
Processing Record 48 of Set 8 | marienburg
Processing Record 49 of Set 8 | bilma
Processing Record 50 of Set 8 | kibakwe
Processing Record 1 of Set 9 | sibolga
Processing Recor

Processing Record 17 of Set 12 | cockburn harbour
City not found. Skipping...
Processing Record 18 of Set 12 | carmen
Processing Record 19 of Set 12 | buriti dos lopes
Processing Record 20 of Set 12 | fonte boa
Processing Record 21 of Set 12 | bereda
Processing Record 22 of Set 12 | miles city
Processing Record 23 of Set 12 | hamilton
Processing Record 24 of Set 12 | pombia
Processing Record 25 of Set 12 | severo-yeniseyskiy
Processing Record 26 of Set 12 | halalo
City not found. Skipping...
Processing Record 27 of Set 12 | koforidua
Processing Record 28 of Set 12 | zadar
Processing Record 29 of Set 12 | svetlaya
Processing Record 30 of Set 12 | sidney
Processing Record 31 of Set 12 | verkhnevilyuysk
Processing Record 32 of Set 12 | george town
Processing Record 33 of Set 12 | bongandanga
Processing Record 34 of Set 12 | puerto del rosario
Processing Record 35 of Set 12 | preobrazheniye
Processing Record 36 of Set 12 | broken hill
Processing Record 37 of Set 12 | tamiahua
Processing Re

Processing Record 2 of Set 16 | punta alta
Processing Record 3 of Set 16 | vetovo
Processing Record 4 of Set 16 | santa marta
Processing Record 5 of Set 16 | vermilion
Processing Record 6 of Set 16 | locri
Processing Record 7 of Set 16 | manchester
Processing Record 8 of Set 16 | fort saint james
Processing Record 9 of Set 16 | bantry
Processing Record 10 of Set 16 | waitara
Processing Record 11 of Set 16 | aripuana
Processing Record 12 of Set 16 | saint-pierre
Processing Record 13 of Set 16 | atar
Processing Record 14 of Set 16 | saryshagan
City not found. Skipping...
Processing Record 15 of Set 16 | bokspits
City not found. Skipping...
Processing Record 16 of Set 16 | hecun
Processing Record 17 of Set 16 | emirdag
Processing Record 18 of Set 16 | navalmoral de la mata
-----------------------------
Data Retrieval Complete      
-----------------------------


In [23]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

706

## Add the weather data to a new DataFrame.

In [24]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [25]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Castro,-24.7911,-50.0119,286.46,98,100,2.00,BR,overcast clouds
1,Albany,42.6001,-73.9662,282.82,75,100,1.22,US,overcast clouds
2,Upernavik,72.7868,-56.1549,267.47,77,100,2.81,GL,overcast clouds
3,Ushuaia,-54.8000,-68.3000,279.96,70,0,1.03,AR,clear sky
4,Barrow,71.2906,-156.7887,269.16,86,100,6.17,US,mist
5,Merauke,-8.4667,140.3333,304.01,65,72,3.41,ID,broken clouds
6,Bluff,-46.6000,168.3333,280.39,86,100,8.92,NZ,light rain
7,Rikitea,-23.1203,-134.9692,295.05,65,89,8.79,PF,overcast clouds
8,Chokurdakh,70.6333,147.9167,269.50,81,99,1.76,RU,overcast clouds
9,Vaini,-21.2000,-175.2000,301.24,69,75,5.66,TO,broken clouds


In [27]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns
city_data_df.columns.tolist()

['City',
 'Lat',
 'Lng',
 'Max Temp',
 'Humidity',
 'Cloudiness',
 'Wind Speed',
 'Country',
 'Current Description']

In [28]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
city_data_df = city_data_df.reindex(columns=["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"])

# Recreate the DataFrame by using the new column order
city_data_df.columns.tolist()

# Display sample data
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Castro,BR,-24.7911,-50.0119,286.46,98,100,2.00,overcast clouds
1,Albany,US,42.6001,-73.9662,282.82,75,100,1.22,overcast clouds
2,Upernavik,GL,72.7868,-56.1549,267.47,77,100,2.81,overcast clouds
3,Ushuaia,AR,-54.8000,-68.3000,279.96,70,0,1.03,clear sky
4,Barrow,US,71.2906,-156.7887,269.16,86,100,6.17,mist
5,Merauke,ID,-8.4667,140.3333,304.01,65,72,3.41,broken clouds
6,Bluff,NZ,-46.6000,168.3333,280.39,86,100,8.92,light rain
7,Rikitea,PF,-23.1203,-134.9692,295.05,65,89,8.79,overcast clouds
8,Chokurdakh,RU,70.6333,147.9167,269.50,81,99,1.76,overcast clouds
9,Vaini,TO,-21.2000,-175.2000,301.24,69,75,5.66,broken clouds


In [31]:
# Display the data types of each column by using the dtypes Pandas function
result = city_data_df.dtypes

print(result)

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object


## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [32]:
# Set the output file name
output_data_file = "WeatherPy_Database.csv"

# Export the city_data DataFrame into a CSV file
city_data_df.to_csv(output_data_file, index_label="City_ID")